In [474]:
import torch
import torcheeg
import numpy as np
from torcheeg.models import EEGNet

#data loader
signal = np.load('idle.npy')
epochs_num = signal.shape[0]
label = np.zeros((epochs_num, 1))
signal = np.vstack([signal, np.load('box.npy')])
label = np.vstack([label, np.zeros((epochs_num, 1)) + 1])
# signal = np.vstack([signal, np.load('pen.npy')])
# label = np.vstack([label, np.zeros((epochs_num, 1)) + 2])

signal = np.vstack([signal, np.load('idle2.npy')])
epochs_num = 30
label = np.vstack([label, np.zeros((epochs_num, 1))])
signal = np.vstack([signal, np.load('box2.npy')])
label = np.vstack([label, np.zeros((epochs_num, 1)) + 1])
# signal = np.vstack([signal, np.load('pen2.npy')])
# label = np.vstack([label, np.zeros((epochs_num, 1)) + 2])

signal = np.vstack([signal, np.load('idle3.npy')])
epochs_num = 25
label = np.vstack([label, np.zeros((epochs_num, 1))])
signal = np.vstack([signal, np.load('box3.npy')])
label = np.vstack([label, np.zeros((epochs_num, 1)) + 1])
# signal = np.vstack([signal, np.load('pen3.npy')])
# label = np.vstack([label, np.zeros((epochs_num, 1)) + 2])



signal = np.expand_dims(signal, 1)
signal.shape, label.shape

((150, 1, 8, 1001), (150, 1))

In [475]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

xt, xv, yt, yv = train_test_split(signal, label, test_size = 0.1, random_state = 42, stratify = label)
print(xt.shape, yt.shape, xv.shape, yv.shape)


(135, 1, 8, 1001) (135, 1) (15, 1, 8, 1001) (15, 1)


In [476]:
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import OneHotEncoder
class capilabDS(Dataset):
    def __init__(self, x, y, device = 'cpu'):
        self.x = x
        self.y = y
        self.device = device
        self.num_class = np.unique(self.y)
        ohe = OneHotEncoder(sparse_output = False)
        self.y = ohe.fit_transform(self.y, self.num_class)
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        
        x = torch.Tensor(self.x[index,:,:,:]).to(self.device, torch.float32)
        y = torch.Tensor(self.y[index, :]).to(self.device, torch.float32)

        return (x, y)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
train_data = capilabDS(xt, yt, device = device)

train_loader = DataLoader(train_data, batch_size=32)

model = EEGNet(chunk_size=1001, num_electrodes=8, num_classes=2, F1 = 8, F2 =16,
D = 3, kernel_1 = 64, kernel_2 = 16, dropout = 0.5).to(device)
criteria = torch.nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr = 1e-4, eps=1e-7)

epochs = 500
epoch_loss = []
for epoch in range(epochs):
    bloss = 0
    bacc = 0
    for i, batch in enumerate(train_loader):
        x, y = batch
        optim.zero_grad()
        yhat = model(x)
        
        loss = criteria(yhat, y)
        loss.backward()
        optim.step()
    
        bloss += loss.item()
        pred = np.argmax(nn.functional.softmax(yhat, 1).detach().cpu().numpy(), 1)
        target = np.argmax(y.detach().cpu().numpy(), 1)
        
        bacc += np.sum(pred == target)
    
    print(f"Epoch {epoch + 1:3d} | loss {bloss / len(train_loader.dataset)} - accuracy {bacc / len(train_loader.dataset)}")


Epoch   1 | loss 0.025657398612410932 - accuracy 0.5259259259259259
Epoch   2 | loss 0.025673809757939092 - accuracy 0.4222222222222222
Epoch   3 | loss 0.025667112844961662 - accuracy 0.5185185185185185
Epoch   4 | loss 0.025642524383686208 - accuracy 0.4074074074074074
Epoch   5 | loss 0.02564128416555899 - accuracy 0.48148148148148145
Epoch   6 | loss 0.025656708523079202 - accuracy 0.5037037037037037
Epoch   7 | loss 0.025671517848968504 - accuracy 0.4740740740740741
Epoch   8 | loss 0.0256521154333044 - accuracy 0.5851851851851851
Epoch   9 | loss 0.025633702013227674 - accuracy 0.48148148148148145
Epoch  10 | loss 0.025642739401923284 - accuracy 0.5259259259259259
Epoch  11 | loss 0.025654402485600223 - accuracy 0.48148148148148145
Epoch  12 | loss 0.025660877757602268 - accuracy 0.5407407407407407
Epoch  13 | loss 0.02563913972289474 - accuracy 0.5333333333333333
Epoch  14 | loss 0.02565738006874367 - accuracy 0.45925925925925926
Epoch  15 | loss 0.0256022987542329 - accuracy 0.

In [485]:
#test
device = 'cuda' if torch.cuda.is_available() else 'cpu'
test_data = capilabDS(xv, yv, device = device)

test_loader = DataLoader(test_data, batch_size=64, shuffle = True)
bacc = 0
with torch.no_grad():
    for i, batch in enumerate(test_loader):
        x, y = batch
        yhat = model(x)

        pred = np.argmax(nn.functional.softmax(yhat, 1).detach().cpu().numpy(), 1)
        target = np.argmax(y.detach().cpu().numpy(), 1)
        bacc += np.sum(pred == target)
        print(target, pred)
    print(f"accuracy {bacc / len(test_loader.dataset) * 100}%")
        

[1 0 0 0 0 0 1 1 1 1 0 0 0 1 1] [1 0 0 0 0 0 1 1 1 1 0 0 0 1 0]
accuracy 93.33333333333333%
